In [1]:
import pandas as pd
import os
from dataclasses import dataclass, field
import sys
sys.path.insert(0, '/home/jovyan/work')
sys.path.insert(1, '/home/jovyan/work/src')
sys.path.insert(2, '/home/jovyan/work/src/service')

from service import bfs_cases_db_service as bfs_db

from py.global_configs import *
from py.normalize import normalize

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:
# check all the file name

FILES_TO_ANALYZE.keys()


dict_keys(['Hirslanden Salem 2017', 'Hirslanden Beau Site 2017', 'Hirslanden Linde 2017', 'Hirslanden Linde 2018', 'Hirslanden Salem 2018', 'Hirslanden Beau Site 2018', 'Winterthur 2017', 'Winterthur 2018', 'Winterthur 2019', 'Winterthur 2020', 'USZ 2018', 'USZ 2019', 'USZ 2019_30_04_2020', 'KSSG 2021'])

In [3]:
file = FILES_TO_ANALYZE['KSSG 2021']
file

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/KSSG_2021.xlsx', hospital_name_db='KSSG HQ', year='2021', sheets=['Änderungen_KSSG_2021'])

In [4]:

df_revised_case_d2d = normalize(file, 0)

Read 25 cases for KSSG HQ 2021
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
pd                  string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


In [5]:
df_revised_case_d2d.head()

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,1112425128,F1B1A46BC1FEF112,M,56,5,3,H41C,K831,M100,"K8031,K8510,D6959",K831,<NA>,<NA>
1,1112427844,59106C27B3536BE5,M,64,9,3,T62C,R5088,M100,"I691,G819",<NA>,<NA>,<NA>
2,1112441515,BB0033F7EF9CCB1F,W,70,3,3,R61D,C833,M100,D6110,<NA>,<NA>,<NA>
3,1112445586,73EFDADADF92D6A4,W,93,7,2,F62D,I5001,M100,F050,<NA>,<NA>,<NA>
4,1112426216,FD14EA356F1A32E4,M,78,4,2,E77E,J128,M100,J4412,J4482,<NA>,<NA>


# match to the database


In [6]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

array(['1112425128', '1112427844', '1112441515', '1112445586',
       '1112426216', '1112427071', '1112430217', '1112432613',
       '1112438704', '1112448747', '1112450067', '1112443326',
       '1112448386', '1112454721', '1112482020', '1112465409',
       '1112458975', '1112459353', '1112471385', '1112497209',
       '1112510073', '1112518499', '1112503831', '1112486634',
       '1112488070'], dtype=object)

In [7]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids(revised_case_id)
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl
0,1.152,456439,12,1112425128,F1B1A46BC1FEF112,56,0,M,5,3,0,0,0,2020-12-31,01,2021-01-05,00,H41C,H41,3
1,0.628,456681,12,1112426216,FD14EA356F1A32E4,78,0,M,4,3,0,0,0,2021-01-04,01,2021-01-08,00,E77E,E77,2
2,0.576,456862,12,1112427071,C47C1FF3CA495488,70,0,W,2,3,0,0,0,2021-01-06,01,2021-01-08,00,I75C,I75,2
3,0.571,456987,12,1112427844,59106C27B3536BE5,64,0,M,9,3,0,0,0,2021-01-07,01,2021-01-16,06,T62C,T62,3
4,0.828,457399,12,1112430217,1E6CD6DFCCD600D9,87,0,M,9,3,0,0,0,2021-01-13,01,2021-01-22,00,E77D,E77,3


In [8]:
# 
print('There are {} out of {} revised cases from DtoD that are matched with the database for {} {}'.format(len(revised_case_db), len(revised_case_id), file.hospital_name_db, file.year))

There are 25 out of 25 revised cases from DtoD that are matched with the database for KSSG HQ 2021


In [9]:
# if we find the match cases, then we need to check if the case_id, gender, year....are match